# Met-Encefalina

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import simtk.openmm as mm
import simtk.unit as unit
import simtk.openmm.app as app
import molsysmt as molsysmt
import openmmtools as mmtools
from pdbfixer import PDBFixer
import mdtraj as md
import nglview as nv
from tqdm import tqdm

In [ ]:
plt.style.use('ggplot')

## Definición del sistema

### Campo de fuerzas

In [ ]:
forcefield = app.ForceField('amber99sbildn.xml', 'tip3p.xml')

### Trabajando el PDB original

In [ ]:
system_pdb=PDBFixer(pdbid="1PLX")

In [ ]:
system_pdb.topology

In [ ]:
for chain in system_pdb.topology.chains():
    print("Chain index {} with pdb id {}:".format(chain.id,chain.index))
    for residue in chain.residues():
        print("\t Residue name {}, index {}:".format(residue.name, residue.index))
        for atom in residue.atoms():
            print("\t \t Atom name {}, index {}".format(atom.name, atom.index))

Es posible que el pdb carezca de algunos residuos o átomos puntuales.

In [ ]:
missing_residues     = system_pdb.findMissingResidues()
nonstandard_residues = system_pdb.findNonstandardResidues()
missing_atoms        = system_pdb.findMissingAtoms()

print('Missing residues:')
print(missing_residues)
print('NonStandard residues:')
print(nonstandard_residues)
print('Missing atoms:')
print(missing_atoms)

En caso de encontrar residues no estandard, los podríamos reemplazar con el comando:

In [ ]:
system_pdb.replaceNonstandardResidues()

En este caso no hay falta de átomos o residuos. Si lo hubiera, PDBFix puede añadirlos con el comando:

In [ ]:
system_pdb.addMissingAtoms()

PDBFix puede añadir hidrógenos según pH y solvatar, como se muestra en la siguiente celda. Pero como se recomienda en la documentación de OpenMM lo vamos a hacer con Modeller. Hay que revisar el flujo de trabajo de YANK para ver como protona, solvata e ioniza.

In [ ]:
if False: # Celda sólo para muestra, no para ejecutar
    system_pdb.addMissingHydrogens(pH=7.4)
    system_pdb.addSolvent(Vec3(5, 5, 5)*nanometer, positiveIon='Na+', negativeIon='Cl-', ionicStrength=0.1*molar)

Antes de solvatar y crear el sistema que simularemos con OpenMM, veamos lo que hasta ahora tenemos:

In [ ]:
view=molsysmt.view(system_pdb)
view.clear()
view.add_ball_and_stick("protein")
view

En el caso de que hubíeramos modificado el pdb original descargado directamente del Protein Data Bank, podemos escribir nuestra nueva versión del fichero:

In [ ]:
app.PDBFile.writeFile(system_pdb.topology, system_pdb.positions, open('metenkephalin_fixed.pdb', 'w'))

Podemos echarle un vistazo al fichero recien creado. Las celdas de jupyter interpretan ciertos comandos básicos del bash.

In [ ]:
### Descomenta la siguiente linea para ver el contenido del fichero
#less metenkephalin_fixed.pdb

### Creamos la caja y solvatamos

Podemos crear la caja y solvatar directamente con PDBFixer o con otras herramientas. Pero lo vamos a hacer con otra herramienta muy util también del flujo de trabajo propuesto por OpenMM, `modeller`, para ir conociendola:

In [ ]:
system_modeller = app.Modeller(system_pdb.topology, system_pdb.positions)

In [ ]:
system_modeller.addHydrogens(forcefield,pH=7.4)

In [ ]:
system_modeller.addSolvent(forcefield, model='tip3p', padding=1.4*unit.nanometers,
                           positiveIon='Na+', negativeIon='Cl-', ionicStrength=0.1*unit.molar)

Podemos si queremos, escribir un nuevo pdb del sistema la caja solvatada.

In [ ]:
app.PDBFile.writeFile(system_modeller.topology, system_modeller.positions,
                      open('metenkephalin_solvated.pdb', 'w'))

In [ ]:

#less metenkephalin_solvated.pdb

In [ ]:
system_modeller.__class__

In [ ]:
molsysmt.get_form

Podemos también visualizar el nuevo sistema

In [ ]:
view_solvated=molsysmt.view(system_modeller)
view_solvated.clear()
view_solvated.add_ball_and_stick("protein")
view_solvated.add_surface("water")
view_solvated

# Estado termodinámico

In [ ]:
# Formalismo NVT
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
temperature = 300*unit.kelvin
pressure    = None

# Parámetros de la simulación

In [ ]:
step_size       = 2*unit.femtoseconds
num_steps       = 400000
saving_period   = 100
num_steps_saved = int(num_steps/saving_period)

# Integrador

In [ ]:
friction   = 1.0/unit.picosecond
integrator = mm.LangevinIntegrator(temperature, friction, step_size)

# Plataforma de simulación

In [ ]:
platform_name = 'CPU'  #platform:     A platform (CPU, OpenCL, CUDA, or reference); default is platform=OpenCL"
# for ii in range(mm.Platform.getNumPlatforms()):
#     print(mm.Platform.getPlatform(ii).getName())
platform = mm.Platform.getPlatformByName(platform_name)

# Reporteros y arrays de salida

In [ ]:
num_atoms  = system.topology.getNumAtoms()
times      = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.picoseconds)
positions  = unit.Quantity(np.zeros([num_steps_saved,num_atoms,3], np.float32), unit.angstroms)
velocities = unit.Quantity(np.zeros([num_steps_saved,num_atoms,3], np.float32), unit.angstroms/unit.picosecond)
potential_energies   = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.kilocalories_per_mole)
kinetic_energies     = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.kilocalories_per_mole)

# Condiciones iniciales

In [ ]:
initial_positions  = system.positions
#initial_velocities = None # Las velocidades serán adjudicadas aleatoriamente según la distribución Maxwell-Boltzmann del estado termodinámico

context = mm.Context(system.system, integrator, platform)
context.setPositions(initial_positions)
context.setVelocitiesToTemperature(temperature)

In [ ]:
state = context.getState(getEnergy=True, getPositions=True, getVelocities=True)
times[0] = state.getTime()
positions[0] = state.getPositions()
velocities[0] = state.getVelocities()
kinetic_energies[0]=state.getKineticEnergy()
potential_energies[0]=state.getPotentialEnergy()

# Corriendo la simulación

In [ ]:
for ii in tqdm(range(num_steps_saved)):
    context.getIntegrator().step(saving_period)
    state = context.getState(getEnergy=True, getPositions=True, getVelocities=True)
    times[ii] = state.getTime()
    positions[ii] = state.getPositions()
    velocities[ii] = state.getVelocities()
    kinetic_energies[ii]=state.getKineticEnergy()
    potential_energies[ii]=state.getPotentialEnergy()

# Análisis de resultados

Accediendo a las posiciones:

In [ ]:
atom_index = 10 # Por ejemplo del átomo 10
plt.rcParams['figure.figsize'] = 18, 4
for ii, ylabel in zip(range(3),['X','Y','Z']):
    plt.plot(times,positions[:,atom_index,ii])
    plt.ylabel(ylabel+' ('+str(positions.unit)+')')
    plt.xlabel('time ('+str(times.unit)+')')
    plt.show()

Representando las energías cinética y potencial:

In [ ]:
plt.rcParams['figure.figsize'] = 18, 4
plt.plot(times,kinetic_energies[:])
plt.ylabel('Kinetic Energy ('+str(kinetic_energies.unit)+')')
plt.xlabel('time ('+str(times.unit)+')')
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = 18, 4
plt.plot(times,potential_energies[:])
plt.ylabel('Potential Energy ('+str(potential_energies.unit)+')')
plt.xlabel('time ('+str(times.unit)+')')
plt.show()

Represento la trayectoria centrada y fitteada según los átomos de la cadena principal:

In [ ]:
mdtraj_topology = md.Topology.from_openmm(system.topology)
mdtrajectory    = md.Trajectory(positions/unit.nanometers, mdtraj_topology)

In [ ]:
mdtrajectory = mdtrajectory.center_coordinates()
mdtrajectory = mdtrajectory.superpose(reference=mdtrajectory, frame=0, 
                                      atom_indices=mdtrajectory.topology.select("backbone"))

In [ ]:
view_traj = nv.show_mdtraj(mdtrajectory)
view_traj

Represento el RMSD de la trayectoria fitteada respecto del frame 0 según los átomos de la cadena principal

In [ ]:
rmsd = md.rmsd(mdtrajectory, mdtrajectory, atom_indices=mdtrajectory.topology.select("backbone"))

In [ ]:
plt.rcParams['figure.figsize'] = 18, 4
plt.plot(times,rmsd[:])
plt.ylabel('RMSD (nm)')
plt.xlabel('time ('+str(times.unit)+')')
plt.show()

Represento los pares de ángulos dihedros phi y psi visitados por la trayectoria sobre el plot de ramachandran.

In [ ]:
phi   = md.compute_phi(mdtrajectory)
psi   = md.compute_psi(mdtrajectory)
omega = md.compute_omega(mdtrajectory)

In [ ]:
plt.rcParams['figure.figsize'] = 8, 8
plt.scatter(phi[1],psi[1])
plt.ylabel('PSI')
plt.xlabel('PHI')
plt.ylim(-np.pi,np.pi)
plt.xlim(-np.pi,np.pi)
plt.show()